In [ ]:

import sys
import os
import time
import numpy as np
import dolfin as dl

from scipy.interpolate import griddata

src_path = "../src/"
 
sys.path.append(src_path + 'pde/')
from meshUtilities import get_dirichlet_bc, get_grid_dirichlet_bc # pyright: ignore[reportMissingImports]

sys.path.append(src_path + 'prior/')
from priorSampler import PriorSampler # pyright: ignore[reportMissingImports]

from poissonModel import PoissonModel

sys.path.append(src_path + 'data/')
from dataMethods import DataProcessor # pyright: ignore[reportMissingImports]

# set seed
seed = 1024
np.random.seed(seed)


data_folder = '../../autodl-tmp/data/'
results_dir = data_folder
if not os.path.exists(results_dir):
    os.makedirs(results_dir)


/root/miniconda3/envs/neuralop/lib/python3.12/site-packages/ufl/__init__.py:250: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
#For training
data_setting = ['',0.005,0.2,1,0]
 
name = data_setting[0]
print('Generating data set: {}'.format(name))
prior_ac = data_setting[1]
prior_cc = data_setting[2]
prior_logn_scale = data_setting[3]
prior_logn_translate = data_setting[4]
nx, ny = 50, 50
fe_order = 1
data_prefix = 'Poisson'

# create mesh
mesh = dl.UnitSquareMesh(nx, ny)

# create function spaces
Vm = dl.FunctionSpace(mesh, 'Lagrange', fe_order)
Vu = Vm

# create prior sampler
prior_sampler = PriorSampler(Vm, prior_ac, prior_cc, seed)

# create model
model = PoissonModel(Vm, Vu, prior_sampler, prior_logn_scale, prior_logn_translate, seed)

num_samples = 5000

w_samples = np.zeros((num_samples, model.m_dim))
m_samples = np.zeros((num_samples, model.m_dim))
u_samples = np.zeros((num_samples, model.u_dim))

w, m, u = model.empty_m(), model.empty_m(), model.empty_u()

for i in range(num_samples):

    start_time = time.perf_counter()

    # draw from gaussian prior
    if i == 0:
        w = prior_sampler.mean.copy()
    else:
        w = prior_sampler(w)[0]
    
    # transform w to m
    m = model.transform_gaussian_pointwise(w, m)
    
    # if m is transformed above, we do not need to transform it again
    u = model.solveFwd(u, m, transform_m = False)

    # save
    w_samples[i, :] = w
    m_samples[i, :] = m
    u_samples[i, :] = u

    end_time = time.perf_counter()
    sample_time = end_time - start_time

    if i % 100 == 0:
        print('Sample {:4d} took {:.3f} seconds'.format(i, sample_time))

print(m_samples.shape, m_samples.shape, u_samples.shape)

proj_w_dim, proj_m_dim, proj_u_dim = 100, 100, 100
tol = 1.e-9
w_mean = np.mean(w_samples, axis = 0)
m_mean = np.mean(m_samples, axis = 0)
u_mean = np.mean(u_samples, axis = 0)
w_std = np.std(w_samples, axis = 0)
m_std = np.std(m_samples, axis = 0)
u_std = np.std(u_samples, axis = 0)
w_normalized = (w_samples - w_mean) / (w_std + tol)
m_normalized = (m_samples - m_mean) / (m_std + tol)
u_normalized = (u_samples - u_mean) / (u_std + tol)
w_SVD, w_s, _ = np.linalg.svd(w_normalized.T, full_matrices = False)
m_SVD, m_s, _ = np.linalg.svd(m_normalized.T, full_matrices = False)
u_SVD, u_s, _ = np.linalg.svd(u_normalized.T, full_matrices = False)

u_mesh_dirichlet_boundary_nodes = get_dirichlet_bc(model.is_point_on_dirichlet_boundary, model.u_nodes)
np.savez(results_dir + data_prefix + f'_samples_{name}.npz', \
        w_samples = w_samples, \
        m_samples = m_samples, \
        u_samples = u_samples, \
        num_samples = num_samples, \
        m_dim = model.m_dim, u_dim = model.u_dim, \
        fe_order = fe_order, \
        nx = nx, ny = ny, \
        prior_ac = prior_ac, \
        prior_cc = prior_cc, \
        prior_alpham = prior_logn_scale, \
        prior_betam = prior_logn_translate, \
        u_mesh_nodes = model.u_nodes, \
        m_mesh_nodes = model.m_nodes, \
        u_mesh_elements = model.Vu.mesh().cells(), \
        m_mesh_elements = model.Vm.mesh().cells(), \
        u_mesh_dirichlet_boundary_nodes = u_mesh_dirichlet_boundary_nodes, \
        w_SVD = w_SVD, w_s = w_s, \
        m_SVD = m_SVD, m_s = m_s, \
        u_SVD = u_SVD, u_s = u_s
        )

num_grid_x, num_grid_y = 51, 51
print('Generating FNO data for grid sizes ({}, {})'.format(num_grid_x, num_grid_y))

# get grid coordinates
grid_x, grid_y = np.meshgrid(np.linspace(0, 1, num_grid_x), np.linspace(0, 1, num_grid_y), indexing='ij')

# load data
data_load = np.load(results_dir + data_prefix + f'_samples_{name}.npz')

m_nodes = data_load['m_mesh_nodes']
u_nodes = data_load['u_mesh_nodes']

w_samples = data_load['w_samples']
m_samples = data_load['m_samples']
u_samples = data_load['u_samples']

num_samples = m_samples.shape[0]

# get indices of grid points on the boundary
u_grid_dirichlet_boundary_nodes = get_grid_dirichlet_bc(model.is_point_on_dirichlet_boundary, grid_x, grid_y)

# interpolate samples on grid
grid_w_samples = np.zeros((num_samples, num_grid_x, num_grid_y))
grid_m_samples = np.zeros((num_samples, num_grid_x, num_grid_y))
grid_u_samples = np.zeros((num_samples, num_grid_x, num_grid_y))
for i in range(num_samples):
    start_time = time.perf_counter()
    
    grid_w_samples[i, :, :] = griddata(m_nodes, w_samples[i, :], (grid_x, grid_y), method='linear')
    grid_m_samples[i, :, :] = griddata(m_nodes, m_samples[i, :], (grid_x, grid_y), method='linear')
    grid_u_samples[i, :, :] = griddata(u_nodes, u_samples[i, :], (grid_x, grid_y), method='linear')

    # explicity set boundary points to zero (not necessary as the solution of PDE is zero on the boundary)
    grid_u_samples[i, u_grid_dirichlet_boundary_nodes[:,0], u_grid_dirichlet_boundary_nodes[:,1]] = 0.0

    end_time = time.perf_counter()
    sample_time = end_time - start_time

    if i % 100 == 0:
        print('Sample {:4d} took {:.3f} seconds'.format(i, sample_time))

print(grid_m_samples.shape, grid_u_samples.shape)

np.savez(results_dir + data_prefix + f'_FNO_samples_{name}.npz', \
        grid_w_samples = grid_w_samples, \
        grid_m_samples = grid_m_samples, \
        grid_u_samples = grid_u_samples, \
        num_samples = num_samples, \
        num_grid_x = num_grid_x, num_grid_y = num_grid_y, \
        grid_x = grid_x, grid_y = grid_y, \
        u_grid_dirichlet_boundary_nodes = u_grid_dirichlet_boundary_nodes)


In [ ]:
data_type=[['no-ood',0.005,0.2,1,0],['ood-1',0.005,0.2,1,0.03],['ood-2',0.005,0.2,1,0.06],\
           ['ood-3',0.005,0.2,1,0.1],['ood-4',0.005,0.2,1,0.15],['ood-5',0.005,0.2,1,0.2], \
            ['ood-6',0.005,0.2,1,-0.2]]
#1st is for verifying ood detection and used as id data; 2-6th are shifting data and 7th is wrongly sampled (m(x) must be positive in physics)
for data_setting in data_type:
    name = data_setting[0]
    print('Generating data set: {}'.format(name))
    prior_ac = data_setting[1]
    prior_cc = data_setting[2]
    prior_logn_scale = data_setting[3]
    prior_logn_translate = data_setting[4]
    nx, ny = 50, 50
    fe_order = 1
    data_prefix = 'Poisson'

    # create mesh
    mesh = dl.UnitSquareMesh(nx, ny)

    # create function spaces
    Vm = dl.FunctionSpace(mesh, 'Lagrange', fe_order)
    Vu = Vm

    # create prior sampler
    prior_sampler = PriorSampler(Vm, prior_ac, prior_cc, seed)

    # create model
    model = PoissonModel(Vm, Vu, prior_sampler, prior_logn_scale, prior_logn_translate, seed)
    num_samples = 500

    w_sample = np.zeros((num_samples, model.m_dim))
    m_sample = np.zeros((num_samples, model.m_dim))
    u_sample = np.zeros((num_samples, model.u_dim))

    w, m, u = model.empty_m(), model.empty_m(), model.empty_u()

    for i in range(num_samples):

        start_time = time.perf_counter()

        # draw from gaussian prior
        if i == 0:
            w = prior_sampler.mean.copy()
        else:
            w = prior_sampler(w)[0]
        
        # transform w to m
        m = model.transform_gaussian_pointwise(w, m)
        
        # if m is transformed above, we do not need to transform it again
        u = model.solveFwd(u, m, transform_m = False)

        # save
        w_sample[i, :] = w
        m_sample[i, :] = m
        u_sample[i, :] = u

    print(m_sample.shape, m_sample.shape, u_sample.shape)
    w_samples = np.array([])  # re-initialize to avoid accumulation
    m_samples = np.array([])
    u_samples = np.array([])
    w_samples = np.concatenate((w_samples, w_sample), axis=0) if len(w_samples) else w_sample
    m_samples = np.concatenate((m_samples, m_sample), axis=0) if len(m_samples) else m_sample
    u_samples = np.concatenate((u_samples, u_sample), axis=0) if len(u_samples) else u_sample

    proj_w_dim, proj_m_dim, proj_u_dim = 100, 100, 100
    tol = 1.e-9
    w_mean = np.mean(w_samples, axis = 0)
    m_mean = np.mean(m_samples, axis = 0)
    u_mean = np.mean(u_samples, axis = 0)
    w_std = np.std(w_samples, axis = 0)
    m_std = np.std(m_samples, axis = 0)
    u_std = np.std(u_samples, axis = 0)
    w_normalized = (w_samples - w_mean) / (w_std + tol)
    m_normalized = (m_samples - m_mean) / (m_std + tol)
    u_normalized = (u_samples - u_mean) / (u_std + tol)
    w_SVD, w_s, _ = np.linalg.svd(w_normalized.T, full_matrices = False)
    m_SVD, m_s, _ = np.linalg.svd(m_normalized.T, full_matrices = False)
    u_SVD, u_s, _ = np.linalg.svd(u_normalized.T, full_matrices = False)

    u_mesh_dirichlet_boundary_nodes = get_dirichlet_bc(model.is_point_on_dirichlet_boundary, model.u_nodes)
    np.savez(results_dir + data_prefix + f'_samples_{name}.npz', \
            w_samples = w_samples, \
            m_samples = m_samples, \
            u_samples = u_samples, \
            num_samples = num_samples, \
            m_dim = model.m_dim, u_dim = model.u_dim, \
            fe_order = fe_order, \
            nx = nx, ny = ny, \
            u_mesh_nodes = model.u_nodes, \
            m_mesh_nodes = model.m_nodes, \
            u_mesh_elements = model.Vu.mesh().cells(), \
            m_mesh_elements = model.Vm.mesh().cells(), \
            u_mesh_dirichlet_boundary_nodes = u_mesh_dirichlet_boundary_nodes, \
            w_SVD = w_SVD, w_s = w_s, \
            m_SVD = m_SVD, m_s = m_s, \
            u_SVD = u_SVD, u_s = u_s
            )

    num_grid_x, num_grid_y = 51, 51
    print('Generating FNO data for grid sizes ({}, {})'.format(num_grid_x, num_grid_y))

    # get grid coordinates
    grid_x, grid_y = np.meshgrid(np.linspace(0, 1, num_grid_x), np.linspace(0, 1, num_grid_y), indexing='ij')

    # load data
    data_load = np.load(results_dir + data_prefix + f'_samples_{name}.npz')

    m_nodes = data_load['m_mesh_nodes']
    u_nodes = data_load['u_mesh_nodes'] 

    w_samples = data_load['w_samples']
    m_samples = data_load['m_samples']
    u_samples = data_load['u_samples']

    num_samples = m_samples.shape[0]

    # get indices of grid points on the boundary
    u_grid_dirichlet_boundary_nodes = get_grid_dirichlet_bc(model.is_point_on_dirichlet_boundary, grid_x, grid_y)

    # interpolate samples on grid
    grid_w_samples = np.zeros((num_samples, num_grid_x, num_grid_y))
    grid_m_samples = np.zeros((num_samples, num_grid_x, num_grid_y))
    grid_u_samples = np.zeros((num_samples, num_grid_x, num_grid_y))
    for i in range(num_samples):
        start_time = time.perf_counter()
        
        grid_w_samples[i, :, :] = griddata(m_nodes, w_samples[i, :], (grid_x, grid_y), method='linear')
        grid_m_samples[i, :, :] = griddata(m_nodes, m_samples[i, :], (grid_x, grid_y), method='linear')
        grid_u_samples[i, :, :] = griddata(u_nodes, u_samples[i, :], (grid_x, grid_y), method='linear')

        # explicity set boundary points to zero (not necessary as the solution of PDE is zero on the boundary)
        grid_u_samples[i, u_grid_dirichlet_boundary_nodes[:,0], u_grid_dirichlet_boundary_nodes[:,1]] = 0.0

        end_time = time.perf_counter()
        sample_time = end_time - start_time

        if i % 100 == 0:
            print('Sample {:4d} took {:.3f} seconds'.format(i, sample_time))

    print(grid_m_samples.shape, grid_u_samples.shape)

    np.savez(results_dir + data_prefix + f'_FNO_samples_{name}.npz', \
            grid_w_samples = grid_w_samples, \
            grid_m_samples = grid_m_samples, \
            grid_u_samples = grid_u_samples, \
            num_samples = num_samples, \
            num_grid_x = num_grid_x, num_grid_y = num_grid_y, \
            grid_x = grid_x, grid_y = grid_y, \
            u_grid_dirichlet_boundary_nodes = u_grid_dirichlet_boundary_nodes)




Generating data set: ood-0.1
(500, 2601) (500, 2601) (500, 2601)
Generating FNO data for grid sizes (51, 51)
Generating data set: ood-0.2
(500, 2601) (500, 2601) (500, 2601)
Generating FNO data for grid sizes (51, 51)
Generating data set: ood-0.4
(500, 2601) (500, 2601) (500, 2601)
Generating FNO data for grid sizes (51, 51)
Generating data set: ood-0.8
(500, 2601) (500, 2601) (500, 2601)
Generating FNO data for grid sizes (51, 51)
Generating data set: ood-1.6
(500, 2601) (500, 2601) (500, 2601)
Generating FNO data for grid sizes (51, 51)
Generating data set: ood--0.1
(500, 2601) (500, 2601) (500, 2601)
Generating FNO data for grid sizes (51, 51)
